In [2]:
import requests
import pandas as pd
import numpy as np
import lxml.html as lh

In [3]:
#Get webpage and validate response
#website is updated daily with reported cases and deaths
url='https://www.michigan.gov/coronavirus/0,9753,7-406-98163-520743--,00.html'
response = requests.get(url)
response

<Response [200]>

In [4]:
#Scrape the table
doc = lh.fromstring(response.content)
tr_elements=doc.xpath('//tr')




In [5]:
#Get headers
col=[]
i=0

for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print ('%d:"%s"'%(i,name.strip()))
        col.append((name.strip(),[]))
        


In [6]:
#Since our first row is the header, data is stored on the second row onwards

#Need to fix dynamic updating of the table length
stopcode=False
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    

    
    #i is the index of our column
    i=0
    
    
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().strip() 
        
        
        if data=='Total':
            stopcode=True
            break
       
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
    
    if stopcode:
        break

In [7]:
#Make the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

#replace nbsp with a 0
df['Deaths'].replace('',0,inplace=True)
df.describe()


,Cases,Deaths
count,34.000000,34.000000
mean,30.441176,0.235294
std,77.018373,0.740959
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,7.750000,0.000000
max,325.000000,4.000000


In [8]:
total_cases=df['Cases'].sum()
print (total_cases)

1035


In [9]:
total_deaths=df['Deaths'].sum()
print (total_deaths)

8


In [10]:
#Note that Detroit is reported seperate from Wayne County
df

,County,Cases,Deaths
0,Allegan,1,0
1,Barry,1,0
2,Bay,1,0
3,Berrien,3,0
4,Calhoun,2,0
5,Charlevoix,2,0
6,Clare,1,0
7,Clinton,2,0
8,Detroit City,325,4
9,Eaton,2,0


In [11]:
#Add a column indicating which counties/cities are in market
df['In Market']=0
df.loc[df['County'].isin(['Detroit City','Lapeer','Macomb','Monroe','Oakland','Genesee','Wayne','St. Clair']),['In Market']]=1

In [264]:
df

,County,Cases,Deaths,In Market
0,Allegan,1,0,0
1,Barry,1,0,0
2,Bay,1,0,0
3,Berrien,3,0,0
4,Calhoun,2,0,0
5,Charlevoix,2,0,0
6,Clare,1,0,0
7,Clinton,2,0,0
8,Detroit City,325,4,1
9,Eaton,2,0,0


In [12]:
df.groupby(['In Market'])['Cases','Deaths'].sum()

<ipython-input-12-d9640024e09d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(['In Market'])['Cases','Deaths'].sum()


,Cases,Deaths
In Market,,
0,114,1
1,921,7
